In [1]:
import mpisppy.utils.sputils as sputils
import pandas as pd
from utils.functions import extract_tem_min, get_data_fromNSRDB, power_PV_calculation, calculate_WT_power
import numpy as np
from utils.model import create_model
import json
from datetime import datetime

[    0.00] Initializing mpi-sppy


## Dimensionar MR OffGrid con escenarios de irradiancia y velocidad del viento

### Seleccionar ubicación

In [11]:
location = {"lat":6.189, "lon":-67.485, "name":"Puerto Carreño", "year_deterministic":2019}
#location = {"lat":12.583, "lon":-81.706, "name":"San Andrés"}

### Escenario determinista

In [12]:
df, info = get_data_fromNSRDB(location["lat"], location["lon"], location["year_deterministic"])
date_vec = np.vectorize(datetime)
df_index = date_vec(df.Year.values,df.Month.values,df.Day.values, df.Hour.values, df.Minute.values, tzinfo=None)
df.index = df_index
df

,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
2019-01-01 00:00:00,2019,1,1,0,0,0,0,0,1.1,23.4,161.98
2019-01-01 01:00:00,2019,1,1,1,0,0,0,0,1.2,23.0,153.19
2019-01-01 02:00:00,2019,1,1,2,0,0,0,0,1.4,22.7,140.91
2019-01-01 03:00:00,2019,1,1,3,0,0,0,0,1.5,22.4,127.58
2019-01-01 04:00:00,2019,1,1,4,0,0,0,0,1.5,22.1,113.88
...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 19:00:00,2019,12,31,19,0,0,0,0,1.2,26.4,112.35
2019-12-31 20:00:00,2019,12,31,20,0,0,0,0,1.2,25.7,126.07
2019-12-31 21:00:00,2019,12,31,21,0,0,0,0,1.1,25.1,139.46
2019-12-31 22:00:00,2019,12,31,22,0,0,0,0,1.0,24.5,151.92


### Parámetros del modelo

In [13]:
catalogo = pd.read_excel("Catalogo.xlsx", sheet_name=None)
series_data = pd.read_csv("CaseData.csv")

data_model = {}

# PARÁMETROS GEOGRÁFICOS
data_model["lat"] = location["lat"]
data_model["lon"] = location["lon"]

# CANTIDAD DE DATOS TEMPORALES
data_model["len_t"] = 8760

# PARÁMETROS ECONÓMICOS
data_model["interest"] = 0.1
data_model["lifeyears"] = 25

# PARÁMETROS DE LA CARGA ELÉCTRICA
data_model["load_el"] = {}
data_model["load_el"]["active"] = True
if data_model["load_el"]["active"]:    
    data_model["load_el"]["type"] = "variable"
    data_model["load_el"]["value"] = series_data["LOAD_EL"].to_numpy()
    
    # PARÁMETROS DE LA ENERGIA ELÉCTRICA NO SUMINISTRADA
    data_model["load_el"]["ENS_EL"] = {}
    data_model["load_el"]["ENS_EL"]["active"] = True
    data_model["load_el"]["ENS_EL"]["type"] = "fixed"
    data_model["load_el"]["ENS_EL"]["value"] = 16

# PARÁMETROS DE LA CARGA TÉRMICA
data_model["load_th"] = {}
data_model["load_th"]["active"] = True
if data_model["load_th"]["active"]:    
    data_model["load_th"]["type"] = "variable"
    data_model["load_th"]["value"] = series_data["LOAD_TH"].to_numpy()
    
    # PARÁMETROS DE LA ENERGIA TÉRMICA NO SUMINISTRADA
    data_model["load_th"]["ENS_TH"] = {}
    data_model["load_th"]["ENS_TH"]["active"] = True
    data_model["load_th"]["ENS_TH"]["type"] = "fixed"
    data_model["load_th"]["ENS_TH"]["value"] = 5

# PARÁMETROS DE LA CARGA DE REFRIGERACIÓN
data_model["load_cl"] = {}
data_model["load_cl"]["active"] = True
if data_model["load_cl"]["active"]:    
    data_model["load_cl"]["type"] = "variable"
    data_model["load_cl"]["value"] = series_data["LOAD_CL"].to_numpy()
    
    # PARÁMETROS DE LA ENERGIA DE REFRIGERACIÓN NO SUMINISTRADA
    data_model["load_cl"]["ENS_CL"] = {}
    data_model["load_cl"]["ENS_CL"]["active"] = True
    data_model["load_cl"]["ENS_CL"]["type"] = "fixed"
    data_model["load_cl"]["ENS_CL"]["value"] = 3
   


# PARÁMETROS DE LOS INVERSORES
data_model["inverters"] = {}
data_model["inverters"]["active"] = True
if data_model["inverters"]["active"]:
    data_model["inverters"]["type"] = catalogo["Hybrid OnGrid"]
    data_model["inverters"]["flex"] = False


# PARÁMETROS DE LAS BATERÍAS
data_model["batteries"] = {}
data_model["batteries"]["active"] = True
data_model["batteries"]["type"] = catalogo["BattModules"]

# PARÁMETROS DE LOS PANELES SOLARES
data_model["pv_modules"] = {}
data_model["pv_modules"]["active"] = True
if data_model["pv_modules"]["active"]:

    data_model["pv_modules"]["type"] = catalogo["PVModules"]

    Temp_min = extract_tem_min(data_model["lat"], data_model["lon"])
    for k in data_model["pv_modules"]["type"].columns:
        data_model["pv_modules"]["type"].loc['Voc_max', k] = np.round(data_model["pv_modules"]["type"].loc['Voc_STC',k]*(1+(data_model["pv_modules"]["type"].loc['Tc_Voc',k]/100)*(Temp_min-25)),2)


# PARÁMETROS DE LAS TURBINAS EÓLICAS
data_model["windgen"] = {}
data_model["windgen"]["active"] = True
data_model["windgen"]["type"] = catalogo["WindTurbines"]

# PARÁMETROS LAS CALDERAS (BOILERS)
data_model["boilers"] = {}
data_model["boilers"]["active"] = True
data_model["boilers"]["type"] = catalogo["Boilers"]

# PARÁMETROS LOS CHPs
data_model["chps"] = {}
data_model["chps"]["active"] = True
data_model["chps"]["type"] = catalogo["CHPs"]

# PARÁMETROS LOS ENFRIADORES DE ABSORCIÓN
data_model["abs_chillers"] = {}
data_model["abs_chillers"]["active"] = True
data_model["abs_chillers"]["type"] = catalogo["AbsorptionChillers"]

# PARÁMETROS LOS ENFRIADORES ELÉCTRICOS
data_model["el_chillers"] = {}
data_model["el_chillers"]["active"] = True
data_model["el_chillers"]["type"] = catalogo["ElectricChillers"]


# PARÁMETROS DE LA RED ELÉCTRICA
data_model["grid_el"] = {}
data_model["grid_el"]["active"] = True
if data_model["grid_el"]["active"]:
    data_model["grid_el"]["pmax_buy"] = 200
    data_model["grid_el"]["pmax_sell"] = 20

    data_model["grid_el"]["buy_price"] = {}
    data_model["grid_el"]["buy_price"]["type"] = "variable"
    data_model["grid_el"]["buy_price"]["value"] = series_data["C_BUY_EL"].to_numpy()

    data_model["grid_el"]["sell_price"] = {}
    data_model["grid_el"]["sell_price"]["type"] = "variable"
    data_model["grid_el"]["sell_price"]["value"] = series_data["C_SELL_EL"].to_numpy()

    data_model["grid_el"]["av"] = {}
    data_model["grid_el"]["av"]["active"] = False

    if data_model["grid_el"]["av"]["active"]:
        pass
        #data_model["grid_el"]["av"]["value"] = av_scens[scens[2]].to_numpy()

# PARÁMETROS DE LA RED DE GAS
data_model["grid_gas"] = {}
data_model["grid_gas"]["active"] = True
if data_model["grid_gas"]["active"]:
    
    data_model["grid_gas"]["pmax_buy"] = 40
    
    data_model["grid_gas"]["buy_price"] = {}
    data_model["grid_gas"]["buy_price"]["type"] = "variable"
    data_model["grid_gas"]["buy_price"]["value"] = series_data["C_BUY_GAS"].to_numpy()

    data_model["grid_gas"]["av"] = {}
    data_model["grid_gas"]["av"]["active"] = False

    if data_model["grid_gas"]["av"]["active"]:
        pass
        #data_model["grid_gas"]["av"]["value"] = av_scens[scens[2]].to_numpy()



# PARÁMETROS DEL GENERADOR DIESEL
#https://cdn.ade-power.com/assets/pdf/generators/cat/cat-de200gc-data-sheet.pdf
data_model["generator"] = {}
data_model["generator"]["active"] = True
if data_model["generator"]["active"]:
    data_model["generator"]["fuel_cost"] = 0.59
    data_model["generator"]["gen_cost"] = 0
    data_model["generator"]["pmax"] = 175
    data_model["generator"]["fmin"] = 1.45
    data_model["generator"]["fmax"] = 50.8 
    data_model["generator"]["fm"] = 0.282
    data_model["generator"]["gen_OM_cost"] = 3
    data_model["generator"]["min_p_load"] = 20

    data_model["generator"]["av"] = {}
    data_model["generator"]["av"]["active"] = True
    if data_model["generator"]["av"]["active"]:
        data_model["generator"]["av"]["value"] = series_data["AV_DIESEL"].to_numpy()

# PARÁMETROS DEL ÁREA DISPONIBLE
data_model["area"] = {}
data_model["area"]["active"] = False
if data_model["area"]["active"]:
    data_model["area"]["value"] = 20


# PARÁMETROS LÍMITE DE INVERSIÓN
data_model["max_invest"] = {}
data_model["max_invest"]["active"] = False  
#data_model["max_invest"]["value"] = 

# PARÁMETROS DE BONOS DE CARBONO
data_model["environment"] = {}
data_model["environment"]["active"] = True
data_model["environment"]["mu"] = 266.76
data_model["environment"]["Cbono"] = 4.23

## Resolver modelo esocástico

### Importar escenarios creados

In [18]:
abrv_name = ''.join([i for i in location["name"] if i.isupper()])

GHI_scens = pd.read_csv(f'Escenarios/GHI_scens_{abrv_name}.csv')
wind_scens = pd.read_csv(f'Escenarios/wind_scens_{abrv_name}.csv')


with open(f'Escenarios/GHI_prob_{abrv_name}.json') as user_file:
    GHI_prob = json.loads(user_file.read())
with open(f'Escenarios/wind_prob_{abrv_name}.json') as user_file:
    wind_prob = json.loads(user_file.read())



### Declarar función scenario_creator

In [19]:
def scenario_creator(scenario_name):

    scens = scenario_name.split("_")

    df_scen = df.copy()

    df_scen["GHI"] = GHI_scens[scens[0]].to_numpy().astype(float)
    df_scen["Wind Speed"] = wind_scens[scens[1]].to_numpy().astype(float)

    if df_scen.isnull().values.any():
        raise ValueError("NaN detected")

    data_model["pv_modules"]["Pmpp"] = power_PV_calculation(df_scen, data_model["pv_modules"]["type"], 0, 10, data_model["lat"])   
    
    Profiles, Wind_generation = calculate_WT_power(df_scen, data_model["windgen"]["type"], 0.001, 20, info['Elevation'].iloc[0])               
    data_model["windgen"]["generation"] = Wind_generation

    
    model = create_model(data_model)
    sputils.attach_root_node(model, model.FirstStage, [model.Xpvs, model.Xpv, model.XBs, model.XB, model.Bxch, model.XCh, model.XT])
    model._mpisppy_probability = GHI_prob[scens[0]]*wind_prob[scens[1]]
    return model

### Escenarios

In [20]:
escenarios = [f'{key_irr}_{key_wind}' for key_irr in GHI_prob for key_wind in wind_prob]
escenarios

['bad_bad',
 'bad_mean',
 'bad_good',
 'mean_bad',
 'mean_mean',
 'mean_good',
 'good_bad',
 'good_mean',
 'good_good']

### Resolver modelo estocástico

In [21]:
from mpisppy.opt.lshaped import LShapedMethod

bounds = {name: -200000 for name in escenarios}
options = {
    "root_solver": "cplex",
    "sp_solver": "cplex",
    "sp_solver_options" : {"threads" : 12},
    "max_iter": 30,
    "valid_eta_lb": bounds
}

ls = LShapedMethod(options, escenarios, scenario_creator)
result = ls.lshaped_algorithm()


[  461.74] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed:  374.15 Time Spent on Last Master:    0.18 Time Spent Generating Last Cut Set:  373.97 Current Objective: -1800000.00
Current Iteration: 3 Time Elapsed: 1123.22 Time Spent on Last Master:    0.15 Time Spent Generating Last Cut Set:  748.92 Current Objective:    0.00
Current Iteration: 4 Time Elapsed: 1920.92 Time Spent on Last Master:    0.11 Time Spent Generating Last Cut Set:  797.58 Current Objective: 403152.46
Current Iteration: 5 Time Elapsed: 2764.47 Time Spent on Last Master:    0.23 Time Spent Generating Last Cut Set:  843.33 Current Objective: 802596.12
Current Iteration: 6 Time Elapsed: 3467.21 Time Spent on Last Master:    0.19 Time Spent Generating Last Cut Set:  702.55 Current Objective: 1387309.00
Current Iteration: 7 Time Elapsed: 4209.68 Time Spent on Last Master:    0.18 Time Spent Generating Last Cut Set:  742.28 Current Objective: 16572

In [22]:
variables = ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

bad_bad Xpvs[EcoGreen-540W,HYD-20kTL] 0.0
bad_bad Xpvs[EcoGreen-540W,IS-15kW] 0.0
bad_bad Xpvs[Risen-590W,HYD-20kTL] 46.0
bad_bad Xpvs[Risen-590W,IS-15kW] 0.0
bad_bad Xpv[EcoGreen-540W,HYD-20kTL] 0.0
bad_bad Xpv[EcoGreen-540W,IS-15kW] 0.0
bad_bad Xpv[Risen-590W,HYD-20kTL] 1058.0
bad_bad Xpv[Risen-590W,IS-15kW] 0.0
bad_bad XBs['BYD-22.1',HYD-20kTL] 0.0
bad_bad XBs['BYD-22.1',IS-15kW] 0.0
bad_bad XBs[PylonTech-UP5000,HYD-20kTL] 31.0
bad_bad XBs[PylonTech-UP5000,IS-15kW] 0.0
bad_bad XB['BYD-22.1',HYD-20kTL] 0.0
bad_bad XB['BYD-22.1',IS-15kW] 0.0
bad_bad XB[PylonTech-UP5000,HYD-20kTL] 248.0
bad_bad XB[PylonTech-UP5000,IS-15kW] 0.0
bad_bad Bxch[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
bad_bad Bxch[EcoGreen-540W,'BYD-22.1',IS-15kW] 0.0
bad_bad Bxch[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
bad_bad Bxch[EcoGreen-540W,PylonTech-UP5000,IS-15kW] 0.0
bad_bad Bxch[Risen-590W,'BYD-22.1',HYD-20kTL] 0.0
bad_bad Bxch[Risen-590W,'BYD-22.1',IS-15kW] 0.0
bad_bad Bxch[Risen-590W,PylonTech-UP5000,HYD-20

In [23]:
from pyomo.environ import value

VPN_F = np.sum([round(1/np.power(1+data_model["interest"],i),3) for i in np.arange(1,data_model["lifeyears"]+1)])
Init_invest = value(ls.local_scenarios[ls.local_scenario_names[0]].FirstStage)
LCOE = 0
CPN = Init_invest

for i in ls.local_scenarios.keys():
    m = ls.local_scenarios[i]
    CPN += value(m.SecondStage)*m._mpisppy_probability
    LCOE += ((value(m.FirstStage) + value(m.SecondStage))/(sum(m.Carga[t] - m.ENS_EL[t].value + sum(m.PpvG[tch,t].value for tch in m.CH) + m.PTG[t].value for t in m.T)*VPN_F))*m._mpisppy_probability


print(f'Inversión incial: {Init_invest}, CPN: {CPN}, LCOE: {LCOE}')

Inversión incial: 1431615.9, CPN: 1772563.9984243372, LCOE: 0.2504404804781522


In [25]:
ls.tree_solution_available=True

In [26]:
ls.write_tree_solution(f'{abrv_name}_est_res_off')

### Resolver modelo deterministico

In [27]:
abrv_name = ''.join([i for i in location["name"] if i.isupper()])

In [28]:
data_model["pv_modules"]["Pmpp"] = power_PV_calculation(df, data_model["pv_modules"]["type"], 0, 10, data_model["lat"])   

Profiles, Wind_generation = calculate_WT_power(df, data_model["windgen"]["type"], 0.001, 20, info['Elevation'].iloc[0])               
data_model["windgen"]["generation"] = Wind_generation 

In [29]:
def scenario_creator(scenario_name):
    
    model = create_model(data_model)
    sputils.attach_root_node(model, model.FirstStage, [model.Xpvs, model.Xpv, model.XBs, model.XB, model.Bxch, model.XCh, model.XT])
    model._mpisppy_probability = 1
    return model

In [30]:
escenarios = ["main"]

In [31]:
from mpisppy.opt.lshaped import LShapedMethod

bounds = {name: -200000 for name in escenarios}
options = {
    "root_solver": "cplex",
    "sp_solver": "cplex",
    "sp_solver_options" : {"threads" : 12},
    "max_iter": 30,
    "valid_eta_lb": bounds
}

ls = LShapedMethod(options, escenarios, scenario_creator)
result = ls.lshaped_algorithm()

[27368.75] Initializing SPBase
Current Iteration: 1 Time Elapsed:    0.00 Current Objective: -Inf
Current Iteration: 2 Time Elapsed:   38.33 Time Spent on Last Master:    0.21 Time Spent Generating Last Cut Set:   38.12 Current Objective: -200000.00
Current Iteration: 3 Time Elapsed:  111.17 Time Spent on Last Master:    0.13 Time Spent Generating Last Cut Set:   72.71 Current Objective:    0.00
Current Iteration: 4 Time Elapsed:  210.37 Time Spent on Last Master:    0.12 Time Spent Generating Last Cut Set:   99.07 Current Objective: 190241.43
Current Iteration: 5 Time Elapsed:  319.21 Time Spent on Last Master:    0.14 Time Spent Generating Last Cut Set:  108.70 Current Objective: 788091.70
Current Iteration: 6 Time Elapsed:  391.33 Time Spent on Last Master:    0.13 Time Spent Generating Last Cut Set:   71.99 Current Objective: 1203775.31
Current Iteration: 7 Time Elapsed:  458.63 Time Spent on Last Master:    0.14 Time Spent Generating Last Cut Set:   67.16 Current Objective: 133581

In [32]:
variables = ls.gather_var_values_to_rank0()
for ((scen_name, var_name), var_value) in variables.items():
    print(scen_name, var_name, var_value)

main Xpvs[EcoGreen-540W,HYD-20kTL] 0.0
main Xpvs[EcoGreen-540W,IS-15kW] 0.0
main Xpvs[Risen-590W,HYD-20kTL] 46.0
main Xpvs[Risen-590W,IS-15kW] 0.0
main Xpv[EcoGreen-540W,HYD-20kTL] 0.0
main Xpv[EcoGreen-540W,IS-15kW] 0.0
main Xpv[Risen-590W,HYD-20kTL] 1058.0
main Xpv[Risen-590W,IS-15kW] 0.0
main XBs['BYD-22.1',HYD-20kTL] 0.0
main XBs['BYD-22.1',IS-15kW] 0.0
main XBs[PylonTech-UP5000,HYD-20kTL] 32.0
main XBs[PylonTech-UP5000,IS-15kW] 0.0
main XB['BYD-22.1',HYD-20kTL] 0.0
main XB['BYD-22.1',IS-15kW] 0.0
main XB[PylonTech-UP5000,HYD-20kTL] 256.0
main XB[PylonTech-UP5000,IS-15kW] 0.0
main Bxch[EcoGreen-540W,'BYD-22.1',HYD-20kTL] 0.0
main Bxch[EcoGreen-540W,'BYD-22.1',IS-15kW] 0.0
main Bxch[EcoGreen-540W,PylonTech-UP5000,HYD-20kTL] 0.0
main Bxch[EcoGreen-540W,PylonTech-UP5000,IS-15kW] 0.0
main Bxch[Risen-590W,'BYD-22.1',HYD-20kTL] 0.0
main Bxch[Risen-590W,'BYD-22.1',IS-15kW] 0.0
main Bxch[Risen-590W,PylonTech-UP5000,HYD-20kTL] 1.0
main Bxch[Risen-590W,PylonTech-UP5000,IS-15kW] 0.0
main XCh[

In [33]:
from pyomo.environ import value

VPN_F = np.sum([round(1/np.power(1+data_model["interest"],i),3) for i in np.arange(1,data_model["lifeyears"]+1)])
Init_invest = value(ls.local_scenarios[ls.local_scenario_names[0]].FirstStage)
LCOE = 0
CPN = Init_invest

for i in ls.local_scenarios.keys():
    m = ls.local_scenarios[i]
    CPN += value(m.SecondStage)*m._mpisppy_probability
    LCOE += ((value(m.FirstStage) + value(m.SecondStage))/(sum(m.Carga[t] - m.ENS_EL[t].value + sum(m.PpvG[tch,t].value for tch in m.CH) + m.PTG[t].value for t in m.T)*VPN_F))*m._mpisppy_probability


print(f'Inversión incial: {Init_invest}, CPN: {CPN}, LCOE: {LCOE}')

Inversión incial: 1460268.5799999998, CPN: 1655434.1866481751, LCOE: 0.23334776049219885


In [34]:
ls.write_tree_solution(f'{abrv_name}_det_res_off')